#### SAS数据清洗
----
由于数据收集、数据加载、数据处理等引发的数据质量问题，易形成脏数据。 经过数据探索分析，并结合数据场景，发现数据可能存在:

• 重复值;

• 缺失值;

• 异常值(离群点); 

• 噪音数据;

**(PS：业务上，账户信息表的CRED_LIMIT授信额度范围\[0, 5,000,000\])**

(1)重复值

如账户信息表，XACCOUNT账户号、CRED_LIMIT授信额度；

|XACCOUNT|CRED_LIMIT|
| --- | --- |
|0001487730|10,000|
|0001487731|50,000|
|0001487731|50,000|
|0001487732|30,000|

(2)缺失值

如账户信息表，XACCOUNT账户号、CRED_LIMIT授信额度；

|XACCOUNT|CRED_LIMIT|
| --- | --- |
|0001487730|10,000|
|0001487731|.|
|0001487732|50,000|
|0001487733|30,000|

(3)异常值(离群点)

异常值指特殊的离群点，不一定错误。

如账户信息表，XACCOUNT账户号、CRED_LIMIT授信额度；

|XACCOUNT|CRED_LIMIT|
| --- | --- |
|0001487730|10,000|
|0001487731|1,000,000|
|0001487732|50,000|
|0001487733|30,000|

(4)噪音数据

噪音包括错误值或偏离期望的孤立点值。

如账户信息表，XACCOUNT账户号、CRED_LIMIT授信额度；

|XACCOUNT|CRED_LIMIT|
| --- | --- |
|0001487730|10,000|
|0001487731|10,000,000|
|0001487732|50,000|
|0001487733|30,000|

----

**脏数据案例**

In [5]:
OPTIONS COMPRESS = YES;

/* 数据案例 CAR */
DATA 
    CARS1(KEEP=ID MAKE MODEL ORIGIN ETL_DT) 
    CARS2(KEEP=ID MSRP1 MSRP2)
;
FORMAT ETL_DT DATE9. ID $8. MSRP1 MSRP2 DOLLAR10.; 
SET SASHELP.CARS;
ID = COMPRESS("CAR9"||PUT(_N_,Z4.));
ETL_DT = "01APR2019"D;
MSRP1 = MSRP;
MSRP2 = INVOICE;
OUTPUT CARS1;
IF MSRP > 20000 THEN OUTPUT CARS2;
RUN;

PROC SORT DATA = CARS1;BY ID;RUN;
PROC SORT DATA = CARS2;BY ID;RUN;

DATA CARS3 CARS4;
MERGE CARS1(IN=A) CARS2(IN=B);
BY ID;
IF A;
IF _N_ IN (10,26,35,75,104,150) THEN ORIGIN = "";
IF _N_ IN (194) THEN DO;
MAKE = "#$Q@f#q^V";
MODEL = "$%g%^u@ed@#rf";
MSRP1 = .;
MSRP2 = .;
END;

IF _N_ IN (20, 21) 
THEN DO;
    MSRP1 = -1;
END;

IF _N_ IN (40, 41, 42) 
THEN DO;
    MSRP2 = -1;
END;

OUTPUT CARS3;
IF _N_ < 8 THEN OUTPUT CARS4;
RUN;


DATA CARS5;
SET CARS4;
IF _N_ > 4 
THEN DO;
    ETL_DT = "19MAR2019"D;
    MSRP1 = MSRP1 - 432;
    MSRP2 = MSRP2 - 323;
END;
RUN;

DATA CAR;
SET CARS3 CARS5;
RUN;
/* 数据案例 CAR */

PROC DELETE DATA = CARS1 CARS2 CARS3 CARS4 CARS5;
RUN;

PROC SURVEYSELECT 
    DATA = CAR METHOD = SRS N = 20
    OUT = CAR_DEMO;
RUN;

PROC PRINT DATA = CAR_DEMO;
RUN;

In [7]:
PROC SORT DATA = CAR OUT = CAR_DUPK NODUPKEY;
BY ID;
RUN;

PROC SORT DATA = CAR OUT = CAR_UNIK NOUNIQUEKEY;
BY ID;
RUN;

In [9]:
/* DUPKEY */
PROC PRINT DATA = CAR_UNIK;
RUN;

Obs,ETL_DT,ID,Make,Model,Origin,MSRP1,MSRP2
1,01APR2019,CAR90001,Acura,MDX,Asia,"$36,945","$33,337"
2,01APR2019,CAR90001,Acura,MDX,Asia,"$36,945","$33,337"
3,01APR2019,CAR90002,Acura,RSX Type S 2dr,Asia,"$23,820","$21,761"
4,01APR2019,CAR90002,Acura,RSX Type S 2dr,Asia,"$23,820","$21,761"
5,01APR2019,CAR90003,Acura,TSX 4dr,Asia,"$26,990","$24,647"
6,01APR2019,CAR90003,Acura,TSX 4dr,Asia,"$26,990","$24,647"
7,01APR2019,CAR90004,Acura,TL 4dr,Asia,"$33,195","$30,299"
8,01APR2019,CAR90004,Acura,TL 4dr,Asia,"$33,195","$30,299"
9,01APR2019,CAR90005,Acura,3.5 RL 4dr,Asia,"$43,755","$39,014"
10,19MAR2019,CAR90005,Acura,3.5 RL 4dr,Asia,"$43,323","$38,691"


In [10]:
/* 处理重复值 */
PROC SORT DATA = CAR OUT = CAR_ETLDT;
BY ID DESCENDING ETL_DT;
RUN;

PROC SORT DATA = CAR_ETLDT OUT = CARD NODUPKEY;
BY ID;
RUN;

In [14]:
/* BAD */
PROC FREQ DATA = CARD;
TABLES ETL_DT;
RUN;

PROC FREQ DATA = CARD(OBS=10);
TABLES MODEL;
RUN;
/* BAD */

ETL_DT,Frequency,Percent,CumulativeFrequency,CumulativePercent
01APR2019,428,100.00,428,100.00
Model,Frequency,Percent,CumulativeFrequency,CumulativePercent
3.5 RL 4dr,1,10.00,1,10.00
3.5 RL w/Navigation 4dr,1,10.00,2,20.00
A4 1.8T 4dr,1,10.00,3,30.00
A4 3.0 4dr,1,10.00,4,40.00
A41.8T convertible 2dr,1,10.00,5,50.00
MDX,1,10.00,6,60.00
NSX coupe 2dr manual S,1,10.00,7,70.00
RSX Type S 2dr,1,10.00,8,80.00


In [15]:
/* MAKE */
PROC FREQ DATA = CARD NOPRINT;
TABLES MAKE/OUT = F_CAR_MAKE;
RUN;

PROC PRINT DATA = F_CAR_MAKE(OBS=10);
RUN;

Obs,Make,COUNT,PERCENT
1,#$Q@f#q^V,1,0.23364
2,Acura,7,1.63551
3,Audi,19,4.43925
4,BMW,20,4.67290
5,Buick,9,2.10280
6,Cadillac,8,1.86916
7,Chevrolet,27,6.30841
8,Chrysler,15,3.50467
9,Dodge,13,3.03738
10,Ford,23,5.37383


In [16]:
/* MODEL WHERE */
PROC FREQ DATA = CARD NOPRINT;
TABLES MODEL/OUT = F_CAR_MODEL_NOR;
RUN;

PROC FREQ DATA = CARD NOPRINT;
TABLES MODEL/OUT = F_CAR_MODEL_WHT;
WHERE SUBSTR(MODEL,1,1) ^= " ";
RUN;

In [17]:
/* ORIGIN MISSING */
PROC FREQ DATA = CARD NOPRINT;
TABLES ORIGIN/OUT=F_CAR_ORIGIN_M;
RUN;

PROC FREQ DATA = CARD NOPRINT;
TABLES ORIGIN/OUT=F_CAR_ORIGIN_NM MISSING;
RUN;

PROC UNIVARIATE DATA = CARD;
VAR MSRP1 MSRP2;
RUN;

In [19]:
/* 处理 */
DATA CAR_MSRP;
FORMAT IMSRP1 IMSRP2 $4.;
SET CARD;
IF MSRP1 = . THEN IMSRP1 = "MISS";
ELSE IF MSRP1 = -1 THEN IMSRP1 = "NEW";
ELSE IMSRP1 = "NORM";

IF MSRP2 = . THEN IMSRP2 = "MISS";
ELSE IF MSRP2 = -1 THEN IMSRP2= "NEW";
ELSE IMSRP2 = "NORM";
RUN;
/* 处理 */

In [20]:
/* 检查 */
PROC FREQ DATA = CAR_MSRP NOPRINT;
TABLES IMSRP1 * IMSRP2/MISSING OUT = CAR_IMSRP12;
RUN;
/* 检查 */

PROC PRINT DATA = CAR_IMSRP12(OBS=10);
RUN;

Obs,IMSRP1,IMSRP2,COUNT,PERCENT
1,MISS,MISS,99,23.1308
2,NEW,NORM,2,0.4673
3,NORM,NEW,3,0.7009
4,NORM,NORM,324,75.7009


----
补充：

#### SAS四则运算

**(PS：此部分正是引起前绪介绍各种脏数据出现原因)**

已知A=4，B=2，求 

• A+B;

• A-B;

• A*B;

• A/B;

需注意：

• 缺失值 

• 除数为0

In [23]:
DATA DEMO1;
A = 4;
B = 2;
/* 1 A + B*/
A1B = A + B;
A1B_SUM = SUM(A, B);

/* 2 A - B*/
A2B = A - B;
A2B_SUM = SUM(A, -B);

/* 3 A * B*/
A3B = A * B;

/* 4 A / B*/
A4B = A / B;
RUN;


DATA DEMO2;
A = 4;
B = .;
/* 1 A + B*/
A1B = A + B;
A1B_SUM = SUM(A, B);

/* 2 A - B*/
A2B = A - B;
A2B_SUM = SUM(A, -B);

/* 3 A * B*/
A3B = A * B;

/* 4 A / B*/
A4B = A / B;

RUN;



DATA DEMO3;
A = 4;
B = 0;

/* 1 A + B*/
A1B = A + B;
A1B_SUM = SUM(A, B);

/* 2 A - B*/
A2B = A - B;
A2B_SUM = SUM(A, -B);

/* 3 A * B*/
A3B = A * B;

/* 4 A / B*/
A4B = A / B;
RUN;


PROC PRINT DATA = DEMO1;
RUN;
PROC PRINT DATA = DEMO2;
RUN;
PROC PRINT DATA = DEMO3;
RUN;

Obs,A,B,A1B,A1B_SUM,A2B,A2B_SUM,A3B,A4B
1,4,2,6,6,2,2,8,2
Obs,A,B,A1B,A1B_SUM,A2B,A2B_SUM,A3B,A4B
1,4,.,.,4,.,4,.,.
Obs,A,B,A1B,A1B_SUM,A2B,A2B_SUM,A3B,A4B
1,4,0,4,4,4,4,0,.
